## 📦 Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## 📂 Load Data

In [2]:
# Load all datasets
relation = pd.read_csv("data/movie_theater_id_relation.csv")
date_info = pd.read_csv("data/date_info.csv")
booknow_theaters = pd.read_csv("data/booknow_theaters.csv")
cinePOS_booking = pd.read_csv("data/cinePOS_booking.csv")
cinePOS_theaters = pd.read_csv("data/cinePOS_theaters.csv")
booknow_visits = pd.read_csv("data/booknow_visits.csv")
booknow_booking = pd.read_csv("data/booknow_booking.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

print("✅ Data Loaded Successfully")
print(f"booknow_visits shape: {booknow_visits.shape}")
print(f"cinePOS_booking shape: {cinePOS_booking.shape}")
print(f"sample_submission shape: {sample_submission.shape}")

✅ Data Loaded Successfully
booknow_visits shape: (214046, 3)
cinePOS_booking shape: (1641966, 4)
sample_submission shape: (38062, 2)


## 🔗 Data Merging & Integration

In [3]:
# Start with booknow_visits as our base dataset
df = booknow_visits.copy()
df['show_date'] = pd.to_datetime(df['show_date'])

print(f"Starting with booknow_visits: {df.shape}")
print(f"Date range: {df['show_date'].min()} to {df['show_date'].max()}")

Starting with booknow_visits: (214046, 3)
Date range: 2023-01-01 00:00:00 to 2024-02-28 00:00:00


In [4]:
# Merge with booknow_theaters
df = df.merge(booknow_theaters, on='book_theater_id', how='left')
print(f"After merging with booknow_theaters: {df.shape}")

After merging with booknow_theaters: (214046, 7)


In [5]:
# Merge with relation (connects book_theater_id to cine_theater_id)
df = df.merge(relation, on='book_theater_id', how='left')
print(f"After merging with relation: {df.shape}")

After merging with relation: (214046, 8)


In [6]:
# Merge with cinePOS_theaters
df = df.merge(cinePOS_theaters, on='cine_theater_id', how='left')
print(f"After merging with cinePOS_theaters: {df.shape}")

After merging with cinePOS_theaters: (214046, 12)


In [7]:
# Merge with date_info
date_info_prep = date_info.copy()
date_info_prep['show_date'] = pd.to_datetime(date_info_prep['show_date'])
df = df.merge(date_info_prep, on='show_date', how='left')
print(f"After merging with date_info: {df.shape}")

After merging with date_info: (214046, 13)


In [8]:
# Aggregate booknow_booking data
booknow_booking_prep = booknow_booking.copy()
booknow_booking_prep['show_datetime'] = pd.to_datetime(booknow_booking_prep['show_datetime'])
booknow_booking_prep['show_date'] = booknow_booking_prep['show_datetime'].dt.date
booknow_booking_prep['show_date'] = pd.to_datetime(booknow_booking_prep['show_date'])

booknow_booking_agg = booknow_booking_prep.groupby(['book_theater_id', 'show_date']).agg({
    'tickets_booked': ['sum', 'mean', 'count']
}).reset_index()
booknow_booking_agg.columns = ['book_theater_id', 'show_date', 'total_tickets_booknow', 'avg_tickets_booknow', 'num_bookings_booknow']

df = df.merge(booknow_booking_agg, on=['book_theater_id', 'show_date'], how='left')
print(f"After merging with booknow_booking: {df.shape}")

After merging with booknow_booking: (214046, 16)


In [9]:
# Aggregate cinePOS_booking data
cinePOS_booking_prep = cinePOS_booking.copy()
cinePOS_booking_prep['show_datetime'] = pd.to_datetime(cinePOS_booking_prep['show_datetime'])
cinePOS_booking_prep['show_date'] = cinePOS_booking_prep['show_datetime'].dt.date
cinePOS_booking_prep['show_date'] = pd.to_datetime(cinePOS_booking_prep['show_date'])

cinePOS_booking_agg = cinePOS_booking_prep.groupby(['cine_theater_id', 'show_date']).agg({
    'tickets_sold': ['sum', 'mean', 'count']
}).reset_index()
cinePOS_booking_agg.columns = ['cine_theater_id', 'show_date', 'total_tickets_cinepos', 'avg_tickets_cinepos', 'num_sales_cinepos']

df = df.merge(cinePOS_booking_agg, on=['cine_theater_id', 'show_date'], how='left')
print(f"After merging with cinePOS_booking: {df.shape}")

After merging with cinePOS_booking: (214046, 19)


## ⚙️ Feature Engineering

In [10]:
# Sort by theater and date
df = df.sort_values(['book_theater_id', 'show_date']).reset_index(drop=True)

# Extract temporal features
df['year'] = df['show_date'].dt.year
df['month'] = df['show_date'].dt.month
df['day'] = df['show_date'].dt.day
df['day_of_year'] = df['show_date'].dt.dayofyear
df['week_of_year'] = df['show_date'].dt.isocalendar().week
df['quarter'] = df['show_date'].dt.quarter
df['is_weekend'] = df['show_date'].dt.dayofweek.isin([5, 6]).astype(int)
df['is_month_start'] = df['show_date'].dt.is_month_start.astype(int)
df['is_month_end'] = df['show_date'].dt.is_month_end.astype(int)

print(f"After adding temporal features: {df.shape}")

After adding temporal features: (214046, 28)


In [11]:
# Create lag features
lag_days = [1, 2, 3, 7, 14, 28]

for lag in lag_days:
    df[f'audience_lag_{lag}'] = df.groupby('book_theater_id')['audience_count'].shift(lag)

print(f"After adding lag features: {df.shape}")

After adding lag features: (214046, 34)


In [12]:
# Create rolling window statistics
rolling_windows = [7, 14, 28]

for window in rolling_windows:
    df[f'audience_rolling_mean_{window}'] = (
        df.groupby('book_theater_id')['audience_count']
        .transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
    )
    df[f'audience_rolling_std_{window}'] = (
        df.groupby('book_theater_id')['audience_count']
        .transform(lambda x: x.shift(1).rolling(window, min_periods=1).std())
    )
    df[f'audience_rolling_max_{window}'] = (
        df.groupby('book_theater_id')['audience_count']
        .transform(lambda x: x.shift(1).rolling(window, min_periods=1).max())
    )

print(f"After adding rolling features: {df.shape}")

After adding rolling features: (214046, 43)


In [13]:
# Create theater statistics
theater_stats = df.groupby('book_theater_id')['audience_count'].agg([
    'mean', 'median', 'std', 'min', 'max'
]).reset_index()
theater_stats.columns = ['book_theater_id', 'theater_avg_audience', 'theater_median_audience', 
                         'theater_std_audience', 'theater_min_audience', 'theater_max_audience']

df = df.merge(theater_stats, on='book_theater_id', how='left')
print(f"After adding theater statistics: {df.shape}")

After adding theater statistics: (214046, 48)


In [14]:
# Encode categorical variables
from sklearn.preprocessing import LabelEncoder

le_dow = LabelEncoder()
df['day_of_week_encoded'] = le_dow.fit_transform(df['day_of_week'].fillna('Unknown'))

df['theater_type_x'] = df['theater_type_x'].fillna('Unknown')
df['theater_type_y'] = df['theater_type_y'].fillna('Unknown')

le_type_x = LabelEncoder()
le_type_y = LabelEncoder()
df['theater_type_x_encoded'] = le_type_x.fit_transform(df['theater_type_x'])
df['theater_type_y_encoded'] = le_type_y.fit_transform(df['theater_type_y'])

print(f"After encoding categorical features: {df.shape}")

After encoding categorical features: (214046, 51)


## 📊 Data Preparation for Modeling

In [15]:
# Define feature columns
exclude_cols = ['book_theater_id', 'show_date', 'audience_count', 'cine_theater_id', 
                'day_of_week', 'theater_type_x', 'theater_type_y']

feature_cols = [col for col in df.columns if col not in exclude_cols]

print(f"Number of features: {len(feature_cols)}")

Number of features: 44


In [16]:
# Time-based train-validation split
train_end_date = pd.Timestamp('2024-02-14')
val_start_date = pd.Timestamp('2024-02-15')

train_df = df[df['show_date'] <= train_end_date].copy()
val_df = df[df['show_date'] >= val_start_date].copy()

print(f"Training set: {train_df.shape}")
print(f"  Date range: {train_df['show_date'].min()} to {train_df['show_date'].max()}")
print(f"Validation set: {val_df.shape}")
print(f"  Date range: {val_df['show_date'].min()} to {val_df['show_date'].max()}")

Training set: (204074, 51)
  Date range: 2023-01-01 00:00:00 to 2024-02-14 00:00:00
Validation set: (9972, 51)
  Date range: 2024-02-15 00:00:00 to 2024-02-28 00:00:00


In [17]:
# Fill missing values
for col in feature_cols:
    if train_df[col].dtype in ['float64', 'int64']:
        median_val = train_df[col].median()
        train_df[col] = train_df[col].fillna(median_val)
        val_df[col] = val_df[col].fillna(median_val)

X_train = train_df[feature_cols]
y_train = train_df['audience_count']
X_val = val_df[feature_cols]
y_val = val_df['audience_count']

# Fill any remaining NaN
X_train = X_train.fillna(0)
X_val = X_val.fillna(0)

# Convert object columns to numeric
for col in X_train.select_dtypes(include='object').columns:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce').fillna(0)
    X_val[col] = pd.to_numeric(X_val[col], errors='coerce').fillna(0)

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")

X_train shape: (204074, 44)
X_val shape: (9972, 44)


## 🤖 Model Training

In [18]:
# Import ML libraries
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
from xgboost import XGBRegressor

print("✅ ML libraries imported")

✅ ML libraries imported


In [19]:
# Train LightGBM Model
print("Training LightGBM Model...")

lgb_model = lgb.LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=8,
    num_leaves=31,
    random_state=42,
    verbose=-1
)

lgb_model.fit(X_train, y_train)

y_pred_train_lgb = np.clip(lgb_model.predict(X_train), 0, None)
y_pred_val_lgb = np.clip(lgb_model.predict(X_val), 0, None)

train_mae_lgb = mean_absolute_error(y_train, y_pred_train_lgb)
val_mae_lgb = mean_absolute_error(y_val, y_pred_val_lgb)
train_rmse_lgb = np.sqrt(mean_squared_error(y_train, y_pred_train_lgb))
val_rmse_lgb = np.sqrt(mean_squared_error(y_val, y_pred_val_lgb))
val_r2_lgb = r2_score(y_val, y_pred_val_lgb)

print(f"\n{'='*60}")
print("LightGBM Results:")
print(f"{'='*60}")
print(f"Train MAE: {train_mae_lgb:.4f}")
print(f"Val MAE:   {val_mae_lgb:.4f}")
print(f"Train RMSE: {train_rmse_lgb:.4f}")
print(f"Val RMSE:   {val_rmse_lgb:.4f}")
print(f"Val R²:     {val_r2_lgb:.4f}")

Training LightGBM Model...

LightGBM Results:
Train MAE: 13.5349
Val MAE:   13.5418
Train RMSE: 19.6052
Val RMSE:   19.5897
Val R²:     0.6158

LightGBM Results:
Train MAE: 13.5349
Val MAE:   13.5418
Train RMSE: 19.6052
Val RMSE:   19.5897
Val R²:     0.6158


In [20]:
# Train XGBoost Model
print("Training XGBoost Model...")

xgb_model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=8,
    random_state=42,
    verbosity=0
)

xgb_model.fit(X_train, y_train)

y_pred_train_xgb = np.clip(xgb_model.predict(X_train), 0, None)
y_pred_val_xgb = np.clip(xgb_model.predict(X_val), 0, None)

train_mae_xgb = mean_absolute_error(y_train, y_pred_train_xgb)
val_mae_xgb = mean_absolute_error(y_val, y_pred_val_xgb)
train_rmse_xgb = np.sqrt(mean_squared_error(y_train, y_pred_train_xgb))
val_rmse_xgb = np.sqrt(mean_squared_error(y_val, y_pred_val_xgb))
val_r2_xgb = r2_score(y_val, y_pred_val_xgb)

print(f"\n{'='*60}")
print("XGBoost Results:")
print(f"{'='*60}")
print(f"Train MAE: {train_mae_xgb:.4f}")
print(f"Val MAE:   {val_mae_xgb:.4f}")
print(f"Train RMSE: {train_rmse_xgb:.4f}")
print(f"Val RMSE:   {val_rmse_xgb:.4f}")
print(f"Val R²:     {val_r2_xgb:.4f}")

Training XGBoost Model...

XGBoost Results:
Train MAE: 11.0082
Val MAE:   13.3933
Train RMSE: 15.1448
Val RMSE:   19.4217
Val R²:     0.6223

XGBoost Results:
Train MAE: 11.0082
Val MAE:   13.3933
Train RMSE: 15.1448
Val RMSE:   19.4217
Val R²:     0.6223


In [21]:
# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features:")
print("="*60)
print(feature_importance.head(20).to_string(index=False))


Top 20 Most Important Features:
                 feature  importance
          audience_lag_1        1891
    theater_std_audience        1695
             day_of_year        1672
    theater_max_audience        1579
          audience_lag_7        1484
     day_of_week_encoded        1425
          audience_lag_2        1389
          audience_lag_3        1242
         audience_lag_14        1206
    theater_avg_audience        1071
 audience_rolling_mean_7        1070
                     day        1065
         audience_lag_28        1061
 theater_median_audience        1000
audience_rolling_mean_28         990
audience_rolling_mean_14         921
  audience_rolling_max_7         910
  audience_rolling_std_7         805
 audience_rolling_max_28         773
 audience_rolling_std_28         741


## 🎯 Generate Test Predictions

In [22]:
# Prepare test data
test_data = sample_submission[['ID']].copy()
test_data['book_theater_id'] = test_data['ID'].str.rsplit('_', n=1).str[0]
test_data['show_date'] = test_data['ID'].str.rsplit('_', n=1).str[1]
test_data['show_date'] = pd.to_datetime(test_data['show_date'])

print(f"Test data shape: {test_data.shape}")
print(f"Date range: {test_data['show_date'].min()} to {test_data['show_date'].max()}")

Test data shape: (38062, 3)
Date range: 2024-03-01 00:00:00 to 2024-04-22 00:00:00


In [23]:
# Function to create features for test data
def create_features_for_prediction(test_df, historical_df, feature_cols_list):
    """
    Create features for test data using historical data for lag/rolling features
    """
    hist_minimal = historical_df[['book_theater_id', 'show_date', 'audience_count']].copy()
    
    test_df = test_df.copy()
    test_df['audience_count'] = np.nan
    combined = pd.concat([hist_minimal, test_df[['book_theater_id', 'show_date', 'audience_count']]], ignore_index=True)
    combined = combined.sort_values(['book_theater_id', 'show_date']).reset_index(drop=True)
    
    # Merge with all data sources
    combined = combined.merge(booknow_theaters, on='book_theater_id', how='left', suffixes=('', '_bn'))
    combined = combined.merge(relation, on='book_theater_id', how='left')
    combined = combined.merge(cinePOS_theaters, on='cine_theater_id', how='left', suffixes=('_x', '_y'))
    
    date_info_prep = date_info.copy()
    date_info_prep['show_date'] = pd.to_datetime(date_info_prep['show_date'])
    combined = combined.merge(date_info_prep, on='show_date', how='left')
    
    combined = combined.merge(booknow_booking_agg, on=['book_theater_id', 'show_date'], how='left')
    combined = combined.merge(cinePOS_booking_agg, on=['cine_theater_id', 'show_date'], how='left')
    
    # Add temporal features
    combined['year'] = combined['show_date'].dt.year
    combined['month'] = combined['show_date'].dt.month
    combined['day'] = combined['show_date'].dt.day
    combined['day_of_year'] = combined['show_date'].dt.dayofyear
    combined['week_of_year'] = combined['show_date'].dt.isocalendar().week
    combined['quarter'] = combined['show_date'].dt.quarter
    combined['is_weekend'] = combined['show_date'].dt.dayofweek.isin([5, 6]).astype(int)
    combined['is_month_start'] = combined['show_date'].dt.is_month_start.astype(int)
    combined['is_month_end'] = combined['show_date'].dt.is_month_end.astype(int)
    
    # Add lag features
    for lag in [1, 2, 3, 7, 14, 28]:
        combined[f'audience_lag_{lag}'] = combined.groupby('book_theater_id')['audience_count'].shift(lag)
    
    # Add rolling features
    for window in [7, 14, 28]:
        combined[f'audience_rolling_mean_{window}'] = (
            combined.groupby('book_theater_id')['audience_count']
            .transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
        )
        combined[f'audience_rolling_std_{window}'] = (
            combined.groupby('book_theater_id')['audience_count']
            .transform(lambda x: x.shift(1).rolling(window, min_periods=1).std())
        )
        combined[f'audience_rolling_max_{window}'] = (
            combined.groupby('book_theater_id')['audience_count']
            .transform(lambda x: x.shift(1).rolling(window, min_periods=1).max())
        )
    
    # Add theater statistics
    combined = combined.merge(theater_stats, on='book_theater_id', how='left')
    
    # Encode categorical variables
    combined['day_of_week_encoded'] = le_dow.transform(combined['day_of_week'].fillna('Unknown'))
    combined['theater_type_x'] = combined['theater_type_x'].fillna('Unknown')
    combined['theater_type_y'] = combined['theater_type_y'].fillna('Unknown')
    combined['theater_type_x_encoded'] = le_type_x.transform(combined['theater_type_x'])
    combined['theater_type_y_encoded'] = le_type_y.transform(combined['theater_type_y'])
    
    # Extract test rows
    test_features = combined[combined['audience_count'].isna()].copy()
    available_cols = [col for col in feature_cols_list if col in test_features.columns]
    
    return test_features[available_cols]

print("✅ Feature generation function created")

✅ Feature generation function created


In [24]:
# Generate features for test set
print("Generating features for test set...")
X_test = create_features_for_prediction(test_data, df, feature_cols)

# Fill missing values
for col in X_test.columns:
    if X_test[col].dtype in ['float64', 'int64']:
        X_test[col] = X_test[col].fillna(X_train[col].median())

# Convert object columns
for col in X_test.select_dtypes(include='object').columns:
    X_test[col] = pd.to_numeric(X_test[col], errors='coerce').fillna(0)

X_test = X_test.fillna(0)

print(f"Test features shape: {X_test.shape}")
print(f"Missing values: {X_test.isnull().sum().sum()}")

Generating features for test set...
Test features shape: (38062, 44)
Missing values: 0
Test features shape: (38062, 44)
Missing values: 0


In [25]:
# Make predictions
print("Making predictions with both models...")

test_predictions_xgb = np.clip(xgb_model.predict(X_test), 0, None)
test_predictions_lgb = np.clip(lgb_model.predict(X_test), 0, None)

# Ensemble: average of both models
test_predictions_ensemble = (test_predictions_xgb + test_predictions_lgb) / 2

print(f"\nPredictions statistics:")
print(f"  XGBoost - Min: {test_predictions_xgb.min():.2f}, Max: {test_predictions_xgb.max():.2f}, Mean: {test_predictions_xgb.mean():.2f}")
print(f"  LightGBM - Min: {test_predictions_lgb.min():.2f}, Max: {test_predictions_lgb.max():.2f}, Mean: {test_predictions_lgb.mean():.2f}")
print(f"  Ensemble - Min: {test_predictions_ensemble.min():.2f}, Max: {test_predictions_ensemble.max():.2f}, Mean: {test_predictions_ensemble.mean():.2f}")

Making predictions with both models...

Predictions statistics:
  XGBoost - Min: 2.23, Max: 173.47, Mean: 41.35
  LightGBM - Min: 3.33, Max: 142.37, Mean: 39.04
  Ensemble - Min: 2.78, Max: 152.11, Mean: 40.20

Predictions statistics:
  XGBoost - Min: 2.23, Max: 173.47, Mean: 41.35
  LightGBM - Min: 3.33, Max: 142.37, Mean: 39.04
  Ensemble - Min: 2.78, Max: 152.11, Mean: 40.20


In [26]:
# Create submission file
submission = sample_submission[['ID']].copy()
submission['audience_count'] = test_predictions_ensemble.astype(int)

# Save to CSV
submission.to_csv('submission.csv', index=False)

print("="*60)
print("✅ SUBMISSION FILE CREATED!")
print("="*60)
print(f"Filename: submission.csv")
print(f"Shape: {submission.shape}")
print(f"\nFirst 10 predictions:")
print(submission.head(10))
print(f"\nLast 10 predictions:")
print(submission.tail(10))
print(f"\nPredictions distribution:")
print(submission['audience_count'].describe())

✅ SUBMISSION FILE CREATED!
Filename: submission.csv
Shape: (38062, 2)

First 10 predictions:
                      ID  audience_count
0  book_00001_2024-03-01              33
1  book_00001_2024-03-02              51
2  book_00001_2024-03-03              55
3  book_00001_2024-03-04              50
4  book_00001_2024-03-06              34
5  book_00001_2024-03-07              33
6  book_00001_2024-03-08              31
7  book_00001_2024-03-09              50
8  book_00001_2024-03-10              60
9  book_00001_2024-03-11              45

Last 10 predictions:
                          ID  audience_count
38052  book_00829_2024-04-11              15
38053  book_00829_2024-04-12              14
38054  book_00829_2024-04-13              15
38055  book_00829_2024-04-14              18
38056  book_00829_2024-04-15              16
38057  book_00829_2024-04-18              14
38058  book_00829_2024-04-19              14
38059  book_00829_2024-04-20              15
38060  book_00829_2024-04-21 

## 📝 Summary

### Pipeline Overview:
1. **Data Loading**: Loaded 7 datasets (214,046 training records)
2. **Data Merging**: Integrated all data sources based on booknow_visits
3. **Feature Engineering**: Created 44 features including lag, rolling, and temporal features
4. **Model Training**: Trained LightGBM and XGBoost (Validation MAE: ~13.4)
5. **Predictions**: Generated 38,062 predictions using ensemble approach
6. **Output**: Saved to `submission.csv`

✅ **Ready for submission!**